# Microservice JSS

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

Define some constant for plots

In [4]:
LANG="en_US.UTF-8"
LANGUAGE="en_US.UTF-8"
LC_ALL="en_US.UTF-8"

#sns.set(font='Jetbrains Mono') # Set font
facecolor_reactive_global = [48/255, 112/255, 173/255, .85]
edgecolor_reactive_global = [48/255, 112/255, 173/255, 1.]
label_reactive_global = "Reactive Global Scaling"

facecolor_reactive_local = [111/255, 178/255, 228/255, .85]
edgecolor_reactive_local = [111/255, 178/255, 228/255, 1.]
label_reactive_local = "Reactive Local Scaling"

## Reactive Local vs Reactive Global Enron

In [5]:
data_folder = 'K20_k4/'
data_files = ['reactive_local_enron.txt', 'reactive_global_enron.txt']

target_cols = ['Latency (ms)', 'Deployed instances', 'Message Loss']
target_labels = ['Latency (milliseconds)', 'Deployed Microservices', 'Message Loss']
save_labels = ['latency', 'deployed_instaces', 'loss']
steps = [50, 10, 200]

In [ ]:
usecols_global = [0,1,3,7,9,13]
usecols_local = [0,1,3,5,9,13]

column_names = [
    "Time (ms)", 
    "Latency (ms)", 
    "Measured Scale Target",
    "Completed msgs",
    "Message Loss",
    "Deployed instances",
]
dfs = {}
for file in data_files:
    scenario = file.split('_')[1]
    if scenario == 'global': cols = usecols_global
    else: cols = usecols_local
    df = pd.read_csv(
            data_folder+file, 
            delim_whitespace=True,
            usecols=cols,
            names=column_names,
            skiprows=[])
    dfs[scenario] = df
        
for target_col, target_label, save_label, step in zip(target_cols, target_labels, save_labels, steps):
    x = dfs['global']['Time (ms)']
    y_msgs = dfs['global']['Measured Scale Target']
    y_global = dfs['global'][target_col]
    y_local = dfs['local'][target_col]
    fig = plt.figure(figsize=[14,10])
    ax = plt.subplot(111)
    ax_background = ax.twinx()

    width = 2.7

    print(dfs["global"]["Deployed instances"])

    
    if save_label == 'latency':
        up = 10000
        down = -1
    elif save_label == 'deployed_instaces':
        up = 60
        down = -1
    else:
        up = 2000
        down = -1


    # define plots
    g = ax.bar(
        x, 
        y_global, 
        linestyle='solid', 
        facecolor=facecolor_reactive_global,
        edgecolor=edgecolor_reactive_global,
        linewidth=1.2,
        label=label_reactive_global,
        zorder=1,
        width=width,
    )
    l = ax.bar(
        x+width+0.35, 
        y_local, 
        linestyle='solid', 
        facecolor=facecolor_reactive_local,
        edgecolor=edgecolor_reactive_local,
        linewidth=1.2,
        label=label_reactive_local, 
        zorder=100,
        width=width,
    )


    # define background plot
    arrived_msgs = ax_background.plot(x[1:], y_msgs[1:], linestyle='dashed', label='Actual workload', color='red', zorder=1000, alpha=.55)
    # set axis limits
    ax.set_yscale('linear')
    ax.set_xlim(-5, 206)
    ax.set_ylim(down, up)

    # set ticks and labels
    ax.set_ylabel(target_label, fontsize=13.5)
    ax.set_xlabel('Time (seconds)', fontsize=13.5)
    ax.set_xticks(range(0, 600, 50))
    ax.set_xticklabels([str(i) for i in range(0, 600, 50)])
   
    # background ticks and labels
    ax_background.set_yticks(range(0, 250, 40))
    ax_background.grid(False)
    ax_background.yaxis.set_label_coords(1.05, 0.5)
    ax_background.set_ylabel('Workload (number of inbound messages)')

    # legend settings
    ax.legend(loc='upper left', ncol=1, frameon=True, fontsize=13.5)
    ax_background.legend(loc='upper right', ncol=1, frameon=True, fontsize=13.5)

    fig.savefig(f'./save_files/{data_folder}{save_label}_reactiveL_vs_reactiveG.pdf', dpi=300, format='pdf', bbox_inches='tight')


0      7
1      7
2      7
3      7
4      7
5      7
6     11
7     17
8     22
9     22
10    22
11    24
12    25
13    26
14    28
15    31
16    31
17    32
18    33
19    33
20    34
21    34
22    34
23    34
24    34
25    34
26    34
27    34
28    34
29    34
30    34
31    34
32    34
33    34
34    37
35    34
36    32
37    32
38    27
39    26
40    26
41    22
42    21
43    21
44    21
45    21
46    21
47    17
48    11
49    10
50     7
51     7
Name: Deployed instances, dtype: int64
0      7
1      7
2      7
3      7
4      7
5      7
6     11
7     17
8     22
9     22
10    22
11    24
12    25
13    26
14    28
15    31
16    31
17    32
18    33
19    33
20    34
21    34
22    34
23    34
24    34
25    34
26    34
27    34
28    34
29    34
30    34
31    34
32    34
33    34
34    37
35    34
36    32
37    32
38    27
39    26
40    26
41    22
42    21
43    21
44    21
45    21
46    21
47    17
48    11
49    10
50     7
51     7
Name: Deployed instances,